In [4]:
import re
import glob
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.Polypeptide import PPBuilder
from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP

In [8]:
short_names_list = [] #list for UniProt identifiers
s_list = [] #list for protein sequences
#the function to convert the list to str
def join_strs(strs):
    result = ''
    for s in strs:
        result += s
    return result
merged_list=list()  
#parsing of associated fasta file
fasta='Homo_sapiens.fasta'
records = list(SeqIO.parse(fasta, "fasta"))
fastas={'id':[],'seq':[],'name':[]}
for i in range(len(records)):
    fastas['id'].append(records[i].id.split(sep='|')[1])
    fastas['seq'].append(str(records[i].seq))
    fastas['name'].append(str(records[i].id.split(sep='|')[2]))
for file in glob.glob(r'UP000005640_9606_HUMAN_v4/*.pdb'): #searching of all pdb files
    try:
        merged='a' #dummy for merged list of secondary structure elements
        x=' '.join(re.findall(r'-(.+?)-', file)) #parsing of UniProt identifier from pdb filename from AlphaFold 
        if x in fastas['id']:
            print(x)
            #running of the DSSP program
            structure = PDBParser().get_structure(x, file)
            ppb=PPBuilder()
            for pp in ppb.build_peptides(structure):
                y=pp.get_sequence()
            new_list=list()
            model = structure[0]
            dssp = DSSP(model, file, dssp="mkdssp")
            for symbol in range(len(str(y))): #retrieving the secondary structure data from DSSP output
                a_key = list(dssp.keys())[symbol] 
                dssp_list=dssp[a_key]
                new_list.append(str(dssp_list[2])) 
            merged=join_strs(new_list) #merging of symbols from DSSP output into the string
            merged_list.append(merged) #appending the data to list of secondary structures
    except:
        print(merged)
    if merged!='a': #only if DSSP running was successful, Uniprot id and primary sequence will be appended
        x=' '.join(re.findall(r'-(.+?)-', file)) 
        short_names_list.append(x)
        s_list.append(str(y))
#creating and exporting to csv the dataframe consisting of UniProt ids, primary and secondary structure
df = pd.DataFrame(columns=['short_names', 'seq', 'ss']) 
df['short_names']=short_names_list
df['seq']=s_list
df['ss']=merged_list
df
df.to_csv('Homo_sapiens.csv')

a
Q6IQ32
